# UnRel

[Website](https://www.di.ens.fr/willow/research/unrel/)

In [7]:
%matplotlib inline
import io
import random
from pathlib import Path
from itertools import zip_longest
from operator import itemgetter
from collections import defaultdict

import cv2
import torch
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image, UnidentifiedImageError
from loguru import logger
from IPython.display import display, Markdown, HTML
from torchvision.ops.boxes import box_iou, nms, batched_nms
from detectron2.structures import BoxMode

from xib.datasets.vrd.metadata import OBJECTS, PREDICATES
from xib.datasets.common import get_exif_orientation

unrel_dir = Path('../../data/unrel/raw')

In [8]:
triplets = [
    tuple(s.replace(' ', '_') for s in t.item().item().split('-'))
    for t in 
    scipy.io.loadmat(unrel_dir.joinpath('annotated_triplets.mat').as_posix())['triplets']
]
print(len(triplets))
print(*triplets[:5], sep='\n')

76
('bike', 'above', 'person')
('building', 'has', 'wheel')
('car', 'above', 'road')
('car', 'in', 'building')
('car', 'in', 'tree')


In [13]:
samples = []

for d in scipy.io.loadmat(unrel_dir.joinpath('annotations.mat').as_posix())['annotations']:
    d = d.squeeze(0).item()
    filename = d['filename'].item().item()
    image_id = d['im_id'].item().item()
    
    img_path = unrel_dir / "images" / filename
    try:
        with Image.open(img_path.as_posix()) as img:
            width, height = img.size
            exif_orientation = get_exif_orientation(img)
    except (FileNotFoundError, UnidentifiedImageError):
        logger.warning(f"Image not found/invalid {img_path}")
        continue

    if exif_orientation is not None:
        logger.warning(
            f"Image {img_path}"
            f"has an EXIF orientation tag, "
            f"check the corresponding boxes!"
        )
        continue
    
    objects = {}
    for o in d['objects'].item():
        o = o.squeeze(0).item()
        class_str = o['category'].item().item()
        class_id = OBJECTS.get_id(class_str.replace(' ', '_'))
        # xmin,ymin,xmax,ymax
        box = tuple(o['box'].squeeze(0).item().squeeze(0))
        
        objects[(class_str, box)] = {
            "category_id": class_id,
            "bbox": box,
            "bbox_mode": BoxMode.XYXY_ABS,
            "box_idx": len(objects),
        }
        
    relations = []
    for r in d['relationships'].item():
        r = r.squeeze(0).item()
        subj_class_str = r['sub'].item().item()
        subj_box = tuple(r['sub_box'].squeeze(0).item().squeeze(0))
        subj_idx = objects[(subj_class_str, subj_box)]['box_idx']
        
        obj_class_str = r['obj'].item().item()
        obj_box = tuple(r['obj_box'].squeeze(0).item().squeeze(0))
        obj_idx = objects[(obj_class_str, obj_box)]['box_idx']
        
        for c in r['rels'].item():
            c = c.item().item().replace(' ', '_')
            relations.append({
                "category_id": PREDICATES.get_id(c),
                "subject_idx": subj_idx,
                "object_idx": obj_idx,
            })
            
    if len(relations) == 0:
        logger.warning(
            f"Image {img_path}" f"has 0 annotated relations!"
        )
    
    samples.append({
        'file_name': filename,
        'image_id': image_id,
        'width': width,
        'height': height,
        'annotations': sorted(objects.values(), key=itemgetter('box_idx')),
        'relations': relations
    })

In [16]:
def data_dict_to_str(d):
    res = []
    res.append(f"#{d['image_id']} {d['file_name']}  (H {d['height']}, W {d['width']})")
    
    res.append('Objects:')
    for o in d['annotations']:
        res.append(f"- {o['box_idx']} {OBJECTS.get_str(o['category_id']):<12} {o['bbox']}")
    
    res.append('Relations:')
    for r in d['relations']:
        res.append(
            f"- {r['subject_idx']} {OBJECTS.get_str(d['annotations'][r['subject_idx']]['category_id']):<10} "
            f"{PREDICATES.get_str(r['category_id']):<15}"
            f"{r['object_idx']} {OBJECTS.get_str(d['annotations'][r['object_idx']]['category_id'])}"
        )
    return '\n'.join(res)
    
for s in random.sample(samples, 10):
    print(data_dict_to_str(s))
    display(Markdown(f"![{s['file_name']}]({unrel_dir / 'images' /s['file_name']})"))
    display(Markdown('---'))

#1122 1122.jpg  (H 584, W 876)
Objects:
- 0 person       (237, 74, 502, 583)
- 1 plane        (247, 8, 381, 137)
- 2 plane        (618, 55, 745, 115)
Relations:
- 0 person     hold           1 plane


![1122.jpg](../../data/unrel/raw/images/1122.jpg)

---

#269 269.jpg  (H 267, W 400)
Objects:
- 0 person       (61, 16, 317, 267)
- 1 shoes        (27, 104, 95, 267)
- 2 shoes        (282, 34, 365, 175)
Relations:
- 0 person     hold           1 shoes
- 0 person     hold           2 shoes


![269.jpg](../../data/unrel/raw/images/269.jpg)

...

![292.jpg](../../data/unrel/raw/images/292.jpg)

---

#916 916.jpg  (H 311, W 460)
Objects:
- 0 person       (288, 36, 380, 284)
- 1 chair        (280, 109, 447, 261)
Relations:
- 0 person     carry          1 chair


![916.jpg](../../data/unrel/raw/images/916.jpg)

---